In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agents-intensive-capstone-project/Hackathon dataset.txt


# Install & Import Libraries

In [2]:
!pip install diffusers transformers accelerate safetensors -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libcugraph-cu12 25.6.0 requ

In [3]:
import os
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("❌ GOOGLE_API_KEY is missing in Kaggle Secrets!")

genai.configure(api_key=GOOGLE_API_KEY)

text_model = genai.GenerativeModel("gemini-2.0-flash-lite")


In [4]:
def get_yt_ideas():
    topic = input("Enter your main topic: ")

    prompt = f"""
    Give 5 unique, clickable YouTube video ideas for the topic: {topic}.
    Make them simple, short, and beginner-friendly.
    """

    try:
        ideas = text_model.generate_content(prompt).text.strip()
        print("\n🎯 Generated Ideas:\n", ideas)
        return ideas
    except Exception as e:
        print("❌ Error generating ideas:", e)

ideas = get_yt_ideas()


StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/sd-turbo",
    torch_dtype=torch.float32
)

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cpu")


In [ ]:
def generate_thumbnail(idea, description):
    prompt = f"""
    YouTube thumbnail design:
    - Bright vibrant colors
    - Big bold text
    - Modern clean layout
    - Suitable emojis (🔥✨📚⚙️💡)
    - Eye-catching style
    Title/Idea: {idea}
    Description: {description}
    """

    print("\n🎨 Generating thumbnail… (5 seconds)...")
    image = pipe(prompt).images[0]
    print("✅ Thumbnail Ready!")
    return image


In [ ]:
chosen_idea = input("\nEnter the idea you want to create a thumbnail for: ")
description = input("Enter the description for the thumbnail: ")

thumbnail = generate_thumbnail(chosen_idea, description)
thumbnail
